# All WAYWT

Script to get all the comments from https://reddit.com/r/femalefashionadvice WAYWT posts from a past chunk of time and write them as a csv.

Fields
1. Username
2. Upvotes
3. Downvotes
4. Comment Date*
5. Comment Time*
6. Comment UNIX timestamp
7. Post Title
8. Post date*
9. Post time*
10. Post UNIX timestamp
11. Comment body wordcount
12. Total number of child comments*
13. Number of second-level comments*

In [149]:
import calendar
import os

from datetime import datetime

import pandas as pd
import praw

from waywt import (
    get_all_submissions_for_month,
    is_waywt,
    print_titles,
    get_all_comments,
    sort_comments_by_upvotes,
    print_top_n_comments,
    create_post_text
)

In [2]:
# Instantiate your read-only reddit instance using PRAW
# You should have these set as environment variables
reddit = praw.Reddit(client_id=os.environ.get('REDDIT_CLIENT_ID'),
                     client_secret=os.environ.get('REDDIT_CLIENT_SECRET'),
                     user_agent=os.environ.get('REDDIT_USER'))

Version 5.0.1 of praw is outdated. Version 5.1.0 was released 23 hours ago.


In [3]:
# read-only subreddit instance
ffa = reddit.subreddit('femalefashionadvice')

In [6]:
def get_all_submissions_since_date(day, month, year):
    """
    Params
    ------
    day: int (1-indexed)
    month: int (1-indexed)
    year: int
    """
    unix_start = datetime.timestamp(datetime(year, month, day))

    return [post for post in ffa.submissions(start=unix_start, extra_query=None)]

In [8]:
def get_comment_tree(comment):
    pass

datetime.datetime(2017, 9, 1, 22, 4, 18, 141744)

In [8]:
all_posts = get_all_submissions_since_date(1, 1, 2017)

In [15]:
def is_waywt_strict(submission):
    title = submission.title.lower()
    return ('waywt' in title) and \
        ('announcement' not in title) and \
        ('top' not in title) and \
        ('theme' not in title) and \
        ('reminder' not in title)

In [16]:
maybe_waywt_posts = [submission for submission in all_posts if is_waywt_strict(submission)]

In [17]:
print_titles(maybe_waywt_posts)

0. WAYWT - August 30, 2017
1. WAYWT - August 27, 2017
2. WAYWT - August 23, 2017
3. WAYWT - August 20, 2017
4. WAYWT - August 16, 2017
5. WAYWT - August 13, 2017
6. WAYWT - August 09, 2017
7. WAYWT - August 06, 2017
8. WAYWT - August 02, 2017
9. WAYWT - July 30, 2017
10. WAYWT - July 26, 2017
11. WAYWT - July 23, 2017
12. WAYWT - July 19, 2017
13. WAYWT - July 16, 2017
14. WAYWT - July 12, 2017
15. WAYWT - July 09, 2017
16. WAYWT - July 05, 2017
17. WAYWT - July 02, 2017
18. WAYWT - June 28, 2017
19. WAYWT - June 25, 2017
20. WAYWT - June 21, 2017
21. WAYWT - June 18, 2017
22. WAYWT - June 14, 2017
23. WAYWT - June 11, 2017
24. WAYWT - June 07, 2017
25. WAYWT - June 04, 2017
26. WAYWT - May 31, 2017
27. WAYWT - May 28, 2017
28. WAYWT - May 24, 2017
29. WAYWT - May 21, 2017
30. WAYWT - May 17, 2017
31. WAYWT - May 14, 2017
32. WAYWT - May 10, 2017
33. WAYWT - May 07, 2017
34. WAYWT - May 03, 2017
35. WAYWT - April 30, 2017
36. Amazon's Echo Look built for WAYWT? (MFA xpost)
37. WAYWT - 

In [24]:
remove = [36, 59, 60]
waywt_posts = []
for i,submission in enumerate(maybe_waywt_posts):
    if i not in remove:
        waywt_posts.append(submission)

In [27]:
sample_post = waywt_posts[0]

top_level_comments = list(sample_post.comments)
all_comments = sample_post.comments.list()

In [28]:
print(len(top_level_comments))
print(len(all_comments))

26
180


In [41]:
def count_top_level_responses(comment):
    return len(comment.replies)

def count_all_responses(comment):
    return len(comment.replies.list())

sample_comment = top_level_comments[0]
print(sample_comment.body)
print()
print(count_top_level_responses(sample_comment))
print(count_all_responses(sample_comment))

21/Toronto

[I've finally finished my summer internship, so of course the first thing I did was get my hair dyed purple!](http://i.imgur.com/tY4tgwR.jpg) (American Apparel, Elliatt, Rick Owens, Stella McCartney)

4
6


In [89]:
def parse_comment(comment):
    return {
        'author': comment.author,
        'posted at (utc unix timestamp)': comment.created_utc,
        'posting utc date': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        'posting utc time': datetime.utcfromtimestamp(comment.created_utc).strftime('%H:%M:%S'),
        'posted at (unix timestamp)': comment.created,
        'wordcount': len(comment.body.split()),
        'upvotes': comment.ups,
        'downvotes': comment.downs,
        'direct reply count': count_top_level_responses(comment),
        'total reply count': count_all_responses(comment)
    }

In [118]:
def parse_post(submission):
    return {
        'title': submission.title,
        'upvotes': submission.ups,
        'downvotes': submission.downs,
        'permalink': 'https://reddit.com' + submission.permalink,
        'posted at (utc unix timestamp)': submission.created_utc,
        'posted at (unix timestamp)': submission.created,
        'posting utc date': datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d'),
        'posting utc time': datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%M-%S'),
        'total comment count': submission.num_comments
    }

In [90]:
parse_comment(sample_comment)

{'author': Redditor(name='nisiamo'),
 'direct reply count': 4,
 'downvotes': 0,
 'posted at (unix timestamp)': 1504140366.0,
 'posted at (utc unix timestamp)': 1504111566.0,
 'posting utc date': '2017-08-30',
 'posting utc time': '16:46:06',
 'total reply count': 6,
 'upvotes': 48,
 'wordcount': 28}

In [119]:
parse_post(sample_post)

{'downvotes': 0,
 'permalink': 'https://reddit.com/r/femalefashionadvice/comments/6x0585/waywt_august_30_2017/',
 'posted at (unix timestamp)': 1504137956.0,
 'posted at (utc unix timestamp)': 1504109156.0,
 'posting utc date': '2017-08-30',
 'posting utc time': '2017-05-56',
 'title': 'WAYWT - August 30, 2017',
 'total comment count': 182,
 'upvotes': 19}

In [131]:
def extract_data(posts):
    all_comments = []
    all_posts = []
    for post in posts:
        post_data = parse_post(post)
        all_posts.append(post_data)
        top_level_comments = list(post.comments)
        for comment in top_level_comments:
            comment_data = parse_comment(comment)
            comment_data.update({'post': post_data['title']})
            all_comments.append(comment_data)
    return all_comments, all_posts

In [132]:
comment_data, post_data = extract_data(waywt_posts)

In [133]:
# list the columns explicitly, otherwise they are written in alphabetical order
df = pd.DataFrame.from_records(comment_data, columns=[
    'author',
    'post',
    'upvotes',
    'downvotes',
    'total reply count',
    'direct reply count',
    'wordcount',
    'posting utc date',
    'posting utc time',
    'posted at (unix timestamp)',
    'posted at (utc unix timestamp)'
])

In [136]:
df[-10:]

,author,post,upvotes,downvotes,total reply count,direct reply count,wordcount,posting utc date,posting utc time,posted at (unix timestamp),posted at (utc unix timestamp)
2728,thecurvynerd,"WAYWT - January 01, 2017",72,0,9,4,31,2017-01-01,16:53:47,1.483318e+09,1.483290e+09
2729,villonious,"WAYWT - January 01, 2017",29,0,6,3,20,2017-01-01,17:32:03,1.483321e+09,1.483292e+09
2730,firstgradeproblemz,"WAYWT - January 01, 2017",27,0,4,2,89,2017-01-01,18:16:55,1.483323e+09,1.483295e+09
2731,ruthannr94,"WAYWT - January 01, 2017",43,0,8,6,64,2017-01-01,19:03:16,1.483326e+09,1.483297e+09
2732,xomandark,"WAYWT - January 01, 2017",29,0,4,2,61,2017-01-01,21:05:29,1.483334e+09,1.483305e+09
2733,xtinamariet,"WAYWT - January 01, 2017",17,0,0,0,135,2017-01-01,18:49:49,1.483325e+09,1.483297e+09
2734,mc1r_variant,"WAYWT - January 01, 2017",15,0,4,2,45,2017-01-01,19:22:55,1.483327e+09,1.483299e+09
2735,ryer123,"WAYWT - January 01, 2017",17,0,6,5,115,2017-01-01,17:10:43,1.483319e+09,1.483291e+09
2736,emaleau,"WAYWT - January 01, 2017",15,0,1,1,112,2017-01-01,18:38:04,1.483325e+09,1.483296e+09
2737,madprudentilla,"WAYWT - January 01, 2017",10,0,8,4,145,2017-01-01,19:36:13,1.483328e+09,1.483299e+09


In [137]:
df.to_csv('waywt_comments.csv', index=False)

In [138]:
posts_df = pd.DataFrame.from_records(post_data, columns=[
    'title',
    'permalink',
    'total comment count',
    'upvotes',
    'downvotes',
    'posting utc date',
    'posting utc time',
    'posted at (unix timestamp)',
    'posted at (utc unix timestamp)'])

In [139]:
posts_df

,title,permalink,total comment count,upvotes,downvotes,posting utc date,posting utc time,posted at (unix timestamp),posted at (utc unix timestamp)
0,"WAYWT - August 30, 2017",https://reddit.com/r/femalefashionadvice/comme...,182,19,0,2017-08-30,2017-05-56,1.504138e+09,1.504109e+09
1,"WAYWT - August 27, 2017",https://reddit.com/r/femalefashionadvice/comme...,238,45,0,2017-08-27,2017-05-47,1.503879e+09,1.503850e+09
2,"WAYWT - August 23, 2017",https://reddit.com/r/femalefashionadvice/comme...,247,54,0,2017-08-23,2017-06-04,1.503533e+09,1.503504e+09
3,"WAYWT - August 20, 2017",https://reddit.com/r/femalefashionadvice/comme...,180,51,0,2017-08-20,2017-05-47,1.503274e+09,1.503245e+09
4,"WAYWT - August 16, 2017",https://reddit.com/r/femalefashionadvice/comme...,187,24,0,2017-08-16,2017-06-04,1.502928e+09,1.502900e+09
5,"WAYWT - August 13, 2017",https://reddit.com/r/femalefashionadvice/comme...,233,26,0,2017-08-13,2017-05-52,1.502669e+09,1.502640e+09
6,"WAYWT - August 09, 2017",https://reddit.com/r/femalefashionadvice/comme...,177,33,0,2017-08-09,2017-05-58,1.502324e+09,1.502295e+09
7,"WAYWT - August 06, 2017",https://reddit.com/r/femalefashionadvice/comme...,227,21,0,2017-08-06,2017-05-45,1.502064e+09,1.502036e+09
8,"WAYWT - August 02, 2017",https://reddit.com/r/femalefashionadvice/comme...,188,20,0,2017-08-02,2017-05-53,1.501719e+09,1.501690e+09
9,"WAYWT - July 30, 2017",https://reddit.com/r/femalefashionadvice/comme...,165,39,0,2017-07-30,2017-05-46,1.501460e+09,1.501431e+09


In [140]:
posts_df.to_csv('waywt_posts_data.csv', index=False)

In [141]:
df.describe()

,upvotes,downvotes,total reply count,direct reply count,wordcount,posted at (unix timestamp),posted at (utc unix timestamp)
count,2738.000000,2738.0,2738.000000,2738.000000,2738.000000,2.738000e+03,2.738000e+03
mean,33.379474,0.0,5.868882,3.153031,69.388970,1.492748e+09,1.492719e+09
std,25.144789,0.0,4.712794,2.271560,56.526554,5.835516e+06,5.835516e+06
min,-8.000000,0.0,0.000000,0.000000,0.000000,1.483315e+09,1.483287e+09
25%,16.000000,0.0,2.000000,1.000000,32.000000,1.487815e+09,1.487786e+09
50%,27.000000,0.0,5.000000,3.000000,58.000000,1.492064e+09,1.492035e+09
75%,44.000000,0.0,8.000000,4.000000,91.750000,1.497257e+09,1.497228e+09
max,201.000000,0.0,38.000000,16.000000,578.000000,1.504241e+09,1.504212e+09


In [144]:
all_top_level_comments = get_all_comments(waywt_posts)

In [145]:
sorted_comments = sort_comments_by_upvotes(all_top_level_comments)

In [153]:
def create_post_text(comments):
    title = "## TOP OF WAYWT"

    # See https://www.reddit.com/wiki/commenting
    table_header = "Rank | Comment | User | Upvotes\n" + \
                   "---- | ---- | ---- | -------\n"

    row = "{rank} | [{post_title}](https://reddit.com{permalink}) | /u/{user} | +{upvotes}\n"

    post = title + '\n' + table_header

    # You could speed this up by refactoring to keep the submission title
    # with each comment, or by just making the permalink text just say 'link'.
    for i, comment in enumerate(comments):
        post += row.format(
            rank=i+1,
            permalink=comment.permalink(),
            user=comment.author,
            upvotes=comment.ups,
            post_title=reddit.submission(id=comment._extract_submission_id()).title
        )
    return post


In [154]:
# whoops, I forgot to assign this to a variable, so the next few cells are just
# trying to get it into proper form for pasting
create_post_text(sorted_comments[:100])

'## TOP OF WAYWT\nRank | Comment | User | Upvotes\n---- | ---- | ---- | -------\n1 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh990an) | /u/full_boyle | +201\n2 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8oszc) | /u/Wasmand | +180\n3 | [WAYWT - August 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6vjyte/waywt_august_23_2017/dm0to3g) | /u/recycledstardust | +172\n4 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejgeyj) | /u/ceruleanmuse | +161\n5 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgyjapg) | /u/full_boyle | +155\n6 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlklu15) | /u/recycledstardust | +153\n7 | [WAYWT - January 18, 2017](https://reddit.com/r/femalefashionadv

In [155]:
text = '## TOP OF WAYWT\nRank | Comment | User | Upvotes\n---- | ---- | ---- | -------\n1 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh990an) | /u/full_boyle | +201\n2 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8oszc) | /u/Wasmand | +180\n3 | [WAYWT - August 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6vjyte/waywt_august_23_2017/dm0to3g) | /u/recycledstardust | +172\n4 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejgeyj) | /u/ceruleanmuse | +161\n5 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgyjapg) | /u/full_boyle | +155\n6 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlklu15) | /u/recycledstardust | +153\n7 | [WAYWT - January 18, 2017](https://reddit.com/r/femalefashionadvice/comments/5oq65a/waywt_january_18_2017/dcl9mmf) | /u/muzzyhoo92 | +153\n8 | [WAYWT - July 12, 2017](https://reddit.com/r/femalefashionadvice/comments/6mux2b/waywt_july_12_2017/dk4nwqm) | /u/madinmadame | +149\n9 | [WAYWT - August 02, 2017](https://reddit.com/r/femalefashionadvice/comments/6r56pb/waywt_august_02_2017/dl2j27y) | /u/ayiyi | +146\n10 | [WAYWT - May 31, 2017](https://reddit.com/r/femalefashionadvice/comments/6efxsi/waywt_may_31_2017/dia0bf6) | /u/ayiyi | +142\n11 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dfflpmn) | /u/niftynonsense | +142\n12 | [WAYWT - February 15, 2017](https://reddit.com/r/femalefashionadvice/comments/5u89ok/waywt_february_15_2017/dds52c9) | /u/sohungy | +140\n13 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh904vg) | /u/saphydoodle | +138\n14 | [WAYWT - July 09, 2017](https://reddit.com/r/femalefashionadvice/comments/6m87z8/waywt_july_09_2017/djzzc91) | /u/full_boyle | +134\n15 | [WAYWT - June 14, 2017](https://reddit.com/r/femalefashionadvice/comments/6h8gqb/waywt_june_14_2017/diwmzaa) | /u/Berryjammy | +134\n16 | [WAYWT - March 19, 2017](https://reddit.com/r/femalefashionadvice/comments/60aqzk/waywt_march_19_2017/df56153) | /u/Wasmand | +132\n17 | [WAYWT - July 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6p22xw/waywt_july_23_2017/dkm2dql) | /u/ayiyi | +131\n18 | [WAYWT - June 21, 2017](https://reddit.com/r/femalefashionadvice/comments/6imsb7/waywt_june_21_2017/dj7epd0) | /u/charity1021 | +131\n19 | [WAYWT - April 19, 2017](https://reddit.com/r/femalefashionadvice/comments/66b4q7/waywt_april_19_2017/dgh3mem) | /u/ayiyi | +131\n20 | [WAYWT - January 29, 2017](https://reddit.com/r/femalefashionadvice/comments/5queyw/waywt_january_29_2017/dd2ap7j) | /u/saphydoodle | +130\n21 | [WAYWT - April 26, 2017](https://reddit.com/r/femalefashionadvice/comments/67p31j/waywt_april_26_2017/dgs5pv3) | /u/babychickyellow | +129\n22 | [WAYWT - February 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5s847m/waywt_february_05_2017/ddd7aw1) | /u/smolherring | +127\n23 | [WAYWT - June 07, 2017](https://reddit.com/r/femalefashionadvice/comments/6fudwa/waywt_june_07_2017/dil4nc7) | /u/shercocked | +125\n24 | [WAYWT - March 29, 2017](https://reddit.com/r/femalefashionadvice/comments/627j7s/waywt_march_29_2017/dfkbhtl) | /u/ayiyi | +123\n25 | [WAYWT - January 22, 2017](https://reddit.com/r/femalefashionadvice/comments/5phooz/waywt_january_22_2017/dcrdygw) | /u/xiaojyao | +123\n26 | [WAYWT - July 19, 2017](https://reddit.com/r/femalefashionadvice/comments/6o9gk9/waywt_july_19_2017/dkfol2x) | /u/emaleau | +122\n27 | [WAYWT - July 16, 2017](https://reddit.com/r/femalefashionadvice/comments/6nmxr1/waywt_july_16_2017/dkaok9t) | /u/BaboonStick | +121\n28 | [WAYWT - April 05, 2017](https://reddit.com/r/femalefashionadvice/comments/63mgd1/waywt_april_05_2017/dfvb11s) | /u/stephamelon | +119\n29 | [WAYWT - April 16, 2017](https://reddit.com/r/femalefashionadvice/comments/65ptu4/waywt_april_16_2017/dgca3ps) | /u/saphydoodle | +118\n30 | [WAYWT - February 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5rgaam/waywt_february_01_2017/dd7dgp6) | /u/shercocked | +118\n31 | [WAYWT - March 12, 2017](https://reddit.com/r/femalefashionadvice/comments/5yzbvg/waywt_march_12_2017/deu7vlo) | /u/saphydoodle | +116\n32 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejetqo) | /u/muzzyhoo92 | +116\n33 | [WAYWT - June 04, 2017](https://reddit.com/r/femalefashionadvice/comments/6f8395/waywt_june_04_2017/dig4ttb) | /u/firstthingmonday | +115\n34 | [WAYWT - April 16, 2017](https://reddit.com/r/femalefashionadvice/comments/65ptu4/waywt_april_16_2017/dgccac5) | /u/goldiehawn | +115\n35 | [WAYWT - April 12, 2017](https://reddit.com/r/femalefashionadvice/comments/64z2hk/waywt_april_12_2017/dg64tqc) | /u/ayiyi | +115\n36 | [WAYWT - January 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5lfogz/waywt_january_01_2017/dbva9nc) | /u/Straightgrizzly22 | +115\n37 | [WAYWT - July 30, 2017](https://reddit.com/r/femalefashionadvice/comments/6qi4u7/waywt_july_30_2017/dkxkcr8) | /u/Straightgrizzly22 | +112\n38 | [WAYWT - July 30, 2017](https://reddit.com/r/femalefashionadvice/comments/6qi4u7/waywt_july_30_2017/dkxu8f0) | /u/sewing_magic | +112\n39 | [WAYWT - July 19, 2017](https://reddit.com/r/femalefashionadvice/comments/6o9gk9/waywt_july_19_2017/dkfyr5x) | /u/PM_ME_ITTYBITTYKITTY | +112\n40 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dffkbn5) | /u/SusannaCarmichael | +112\n41 | [WAYWT - March 22, 2017](https://reddit.com/r/femalefashionadvice/comments/60vi51/waywt_march_22_2017/df9kjdt) | /u/shercocked | +112\n42 | [WAYWT - January 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5lfogz/waywt_january_01_2017/dbvbvwl) | /u/abexaem | +111\n43 | [WAYWT - April 12, 2017](https://reddit.com/r/femalefashionadvice/comments/64z2hk/waywt_april_12_2017/dg640ls) | /u/TwoSizes | +110\n44 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dffoly1) | /u/BlueSpiritBlues | +110\n45 | [WAYWT - August 20, 2017](https://reddit.com/r/femalefashionadvice/comments/6uwiut/waywt_august_20_2017/dlwadct) | /u/wanderedoff | +108\n46 | [WAYWT - August 09, 2017](https://reddit.com/r/femalefashionadvice/comments/6smgup/waywt_august_09_2017/dldx97t) | /u/emaleau | +108\n47 | [WAYWT - May 17, 2017](https://reddit.com/r/femalefashionadvice/comments/6bpmgl/waywt_may_17_2017/dhom2ir) | /u/skyllanyx | +107\n48 | [WAYWT - March 29, 2017](https://reddit.com/r/femalefashionadvice/comments/627j7s/waywt_march_29_2017/dfkebuu) | /u/DrNooooooo | +107\n49 | [WAYWT - March 19, 2017](https://reddit.com/r/femalefashionadvice/comments/60aqzk/waywt_march_19_2017/df4vai9) | /u/somethings_nothings | +107\n50 | [WAYWT - March 15, 2017](https://reddit.com/r/femalefashionadvice/comments/5zk88l/waywt_march_15_2017/deyqcvk) | /u/ayiyi | +107\n51 | [WAYWT - July 02, 2017](https://reddit.com/r/femalefashionadvice/comments/6ktzj8/waywt_july_02_2017/djosxbk) | /u/salted_sky | +106\n52 | [WAYWT - May 24, 2017](https://reddit.com/r/femalefashionadvice/comments/6d3352/waywt_may_24_2017/dhzicjz) | /u/Wasmand | +106\n53 | [WAYWT - February 12, 2017](https://reddit.com/r/femalefashionadvice/comments/5tm3q4/waywt_february_12_2017/ddniypv) | /u/BlueSpiritBlues | +106\n54 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dffmjgs) | /u/sohungy | +105\n55 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejpq5h) | /u/cikayelle | +105\n56 | [WAYWT - May 14, 2017](https://reddit.com/r/femalefashionadvice/comments/6b4mfm/waywt_may_14_2017/dhk4p8t) | /u/maya1987_K | +104\n57 | [WAYWT - May 10, 2017](https://reddit.com/r/femalefashionadvice/comments/6adozz/waywt_may_10_2017/dhdrbqm) | /u/super_cheeky | +104\n58 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8q9sw) | /u/sohungy | +104\n59 | [WAYWT - February 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5rgaam/waywt_february_01_2017/dd72crh) | /u/ayiyi | +104\n60 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgy7gdo) | /u/stephamelon | +103\n61 | [WAYWT - August 09, 2017](https://reddit.com/r/femalefashionadvice/comments/6smgup/waywt_august_09_2017/dle4zlg) | /u/e___e | +102\n62 | [WAYWT - January 22, 2017](https://reddit.com/r/femalefashionadvice/comments/5phooz/waywt_january_22_2017/dcrd132) | /u/mycatispipeyq | +102\n63 | [WAYWT - July 19, 2017](https://reddit.com/r/femalefashionadvice/comments/6o9gk9/waywt_july_19_2017/dkfpjar) | /u/Lillyeth | +101\n64 | [WAYWT - March 19, 2017](https://reddit.com/r/femalefashionadvice/comments/60aqzk/waywt_march_19_2017/df4y28u) | /u/BlueSpiritBlues | +101\n65 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejmbo4) | /u/bean-sprout | +101\n66 | [WAYWT - May 17, 2017](https://reddit.com/r/femalefashionadvice/comments/6bpmgl/waywt_may_17_2017/dhoz9xl) | /u/hiilive | +100\n67 | [WAYWT - August 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6vjyte/waywt_august_23_2017/dm0ztkl) | /u/evrtgurnot | +99\n68 | [WAYWT - June 04, 2017](https://reddit.com/r/femalefashionadvice/comments/6f8395/waywt_june_04_2017/digcg3f) | /u/goldiehawn | +99\n69 | [WAYWT - July 30, 2017](https://reddit.com/r/femalefashionadvice/comments/6qi4u7/waywt_july_30_2017/dkxpliz) | /u/e___e | +98\n70 | [WAYWT - May 17, 2017](https://reddit.com/r/femalefashionadvice/comments/6bpmgl/waywt_may_17_2017/dhoi5sx) | /u/faux-kwah | +98\n71 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgy9bru) | /u/babychickyellow | +97\n72 | [WAYWT - February 15, 2017](https://reddit.com/r/femalefashionadvice/comments/5u89ok/waywt_february_15_2017/ddships) | /u/shercocked | +97\n73 | [WAYWT - July 12, 2017](https://reddit.com/r/femalefashionadvice/comments/6mux2b/waywt_july_12_2017/dk4s5jx) | /u/shercocked | +96\n74 | [WAYWT - June 21, 2017](https://reddit.com/r/femalefashionadvice/comments/6imsb7/waywt_june_21_2017/dj7jsam) | /u/shercocked | +96\n75 | [WAYWT - April 26, 2017](https://reddit.com/r/femalefashionadvice/comments/67p31j/waywt_april_26_2017/dgs74zy) | /u/super_cheeky | +96\n76 | [WAYWT - August 20, 2017](https://reddit.com/r/femalefashionadvice/comments/6uwiut/waywt_august_20_2017/dlw9bei) | /u/full_boyle | +95\n77 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlkc2zc) | /u/Bloopity_bloop_bloop | +95\n78 | [WAYWT - July 16, 2017](https://reddit.com/r/femalefashionadvice/comments/6nmxr1/waywt_july_16_2017/dkapnak) | /u/PM_ME_ITTYBITTYKITTY | +95\n79 | [WAYWT - April 12, 2017](https://reddit.com/r/femalefashionadvice/comments/64z2hk/waywt_april_12_2017/dg65lnf) | /u/LentilBurguer | +95\n80 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejfmyg) | /u/SusannaCarmichael | +95\n81 | [WAYWT - January 29, 2017](https://reddit.com/r/femalefashionadvice/comments/5queyw/waywt_january_29_2017/dd292hu) | /u/BlueSpiritBlues | +95\n82 | [WAYWT - January 25, 2017](https://reddit.com/r/femalefashionadvice/comments/5q3z3z/waywt_january_25_2017/dcw7vbg) | /u/shercocked | +95\n83 | [WAYWT - April 23, 2017](https://reddit.com/r/femalefashionadvice/comments/672zjc/waywt_april_23_2017/dgnl12t) | /u/kcwashere | +94\n84 | [WAYWT - April 23, 2017](https://reddit.com/r/femalefashionadvice/comments/672zjc/waywt_april_23_2017/dgn992o) | /u/niftynonsense | +94\n85 | [WAYWT - August 27, 2017](https://reddit.com/r/femalefashionadvice/comments/6wcypl/waywt_august_27_2017/dm7f7us) | /u/recycledstardust | +93\n86 | [WAYWT - July 02, 2017](https://reddit.com/r/femalefashionadvice/comments/6ktzj8/waywt_july_02_2017/djotysz) | /u/saphydoodle | +93\n87 | [WAYWT - April 02, 2017](https://reddit.com/r/femalefashionadvice/comments/630f8l/waywt_april_02_2017/dfqgxt4) | /u/shercocked | +93\n88 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejnke6) | /u/ahrieru | +93\n89 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8quxg) | /u/SusannaCarmichael | +93\n90 | [WAYWT - February 12, 2017](https://reddit.com/r/femalefashionadvice/comments/5tm3q4/waywt_february_12_2017/ddnksu0) | /u/somethings_nothings | +93\n91 | [WAYWT - January 08, 2017](https://reddit.com/r/femalefashionadvice/comments/5mrhi4/waywt_january_08_2017/dc5xmt3) | /u/kcwashere | +93\n92 | [WAYWT - June 18, 2017](https://reddit.com/r/femalefashionadvice/comments/6i0eei/waywt_june_18_2017/dj2sd9k) | /u/bean-sprout | +92\n93 | [WAYWT - February 22, 2017](https://reddit.com/r/femalefashionadvice/comments/5vjhx0/waywt_february_22_2017/de2np2z) | /u/ayiyi | +92\n94 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgy6owp) | /u/coffee_for_dinner | +91\n95 | [WAYWT - January 18, 2017](https://reddit.com/r/femalefashionadvice/comments/5oq65a/waywt_january_18_2017/dcl6rgj) | /u/nnutella | +91\n96 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlkd2kd) | /u/Lillyeth | +90\n97 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh8yhq9) | /u/coffee_for_dinner | +90\n98 | [WAYWT - April 19, 2017](https://reddit.com/r/femalefashionadvice/comments/66b4q7/waywt_april_19_2017/dgh2hkn) | /u/orlando_furiosa | +90\n99 | [WAYWT - April 19, 2017](https://reddit.com/r/femalefashionadvice/comments/66b4q7/waywt_april_19_2017/dgh2u6c) | /u/tomlizzo | +90\n100 | [WAYWT - March 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5wwseh/waywt_march_01_2017/dedferu) | /u/ayiyi | +90\n'

In [156]:
print(text)

## TOP OF WAYWT
Rank | Comment | User | Upvotes
---- | ---- | ---- | -------
1 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh990an) | /u/full_boyle | +201
2 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8oszc) | /u/Wasmand | +180
3 | [WAYWT - August 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6vjyte/waywt_august_23_2017/dm0to3g) | /u/recycledstardust | +172
4 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejgeyj) | /u/ceruleanmuse | +161
5 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgyjapg) | /u/full_boyle | +155
6 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlklu15) | /u/recycledstardust | +153
7 | [WAYWT - January 18, 2017](https://reddit.com/r/femalefashionadvice/commen

## TOP OF WAYWT
Rank | Comment | User | Upvotes
---- | ---- | ---- | -------
1 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh990an) | /u/full_boyle | +201
2 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8oszc) | /u/Wasmand | +180
3 | [WAYWT - August 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6vjyte/waywt_august_23_2017/dm0to3g) | /u/recycledstardust | +172
4 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejgeyj) | /u/ceruleanmuse | +161
5 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgyjapg) | /u/full_boyle | +155
6 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlklu15) | /u/recycledstardust | +153
7 | [WAYWT - January 18, 2017](https://reddit.com/r/femalefashionadvice/comments/5oq65a/waywt_january_18_2017/dcl9mmf) | /u/muzzyhoo92 | +153
8 | [WAYWT - July 12, 2017](https://reddit.com/r/femalefashionadvice/comments/6mux2b/waywt_july_12_2017/dk4nwqm) | /u/madinmadame | +149
9 | [WAYWT - August 02, 2017](https://reddit.com/r/femalefashionadvice/comments/6r56pb/waywt_august_02_2017/dl2j27y) | /u/ayiyi | +146
10 | [WAYWT - May 31, 2017](https://reddit.com/r/femalefashionadvice/comments/6efxsi/waywt_may_31_2017/dia0bf6) | /u/ayiyi | +142
11 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dfflpmn) | /u/niftynonsense | +142
12 | [WAYWT - February 15, 2017](https://reddit.com/r/femalefashionadvice/comments/5u89ok/waywt_february_15_2017/dds52c9) | /u/sohungy | +140
13 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh904vg) | /u/saphydoodle | +138
14 | [WAYWT - July 09, 2017](https://reddit.com/r/femalefashionadvice/comments/6m87z8/waywt_july_09_2017/djzzc91) | /u/full_boyle | +134
15 | [WAYWT - June 14, 2017](https://reddit.com/r/femalefashionadvice/comments/6h8gqb/waywt_june_14_2017/diwmzaa) | /u/Berryjammy | +134
16 | [WAYWT - March 19, 2017](https://reddit.com/r/femalefashionadvice/comments/60aqzk/waywt_march_19_2017/df56153) | /u/Wasmand | +132
17 | [WAYWT - July 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6p22xw/waywt_july_23_2017/dkm2dql) | /u/ayiyi | +131
18 | [WAYWT - June 21, 2017](https://reddit.com/r/femalefashionadvice/comments/6imsb7/waywt_june_21_2017/dj7epd0) | /u/charity1021 | +131
19 | [WAYWT - April 19, 2017](https://reddit.com/r/femalefashionadvice/comments/66b4q7/waywt_april_19_2017/dgh3mem) | /u/ayiyi | +131
20 | [WAYWT - January 29, 2017](https://reddit.com/r/femalefashionadvice/comments/5queyw/waywt_january_29_2017/dd2ap7j) | /u/saphydoodle | +130
21 | [WAYWT - April 26, 2017](https://reddit.com/r/femalefashionadvice/comments/67p31j/waywt_april_26_2017/dgs5pv3) | /u/babychickyellow | +129
22 | [WAYWT - February 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5s847m/waywt_february_05_2017/ddd7aw1) | /u/smolherring | +127
23 | [WAYWT - June 07, 2017](https://reddit.com/r/femalefashionadvice/comments/6fudwa/waywt_june_07_2017/dil4nc7) | /u/shercocked | +125
24 | [WAYWT - March 29, 2017](https://reddit.com/r/femalefashionadvice/comments/627j7s/waywt_march_29_2017/dfkbhtl) | /u/ayiyi | +123
25 | [WAYWT - January 22, 2017](https://reddit.com/r/femalefashionadvice/comments/5phooz/waywt_january_22_2017/dcrdygw) | /u/xiaojyao | +123
26 | [WAYWT - July 19, 2017](https://reddit.com/r/femalefashionadvice/comments/6o9gk9/waywt_july_19_2017/dkfol2x) | /u/emaleau | +122
27 | [WAYWT - July 16, 2017](https://reddit.com/r/femalefashionadvice/comments/6nmxr1/waywt_july_16_2017/dkaok9t) | /u/BaboonStick | +121
28 | [WAYWT - April 05, 2017](https://reddit.com/r/femalefashionadvice/comments/63mgd1/waywt_april_05_2017/dfvb11s) | /u/stephamelon | +119
29 | [WAYWT - April 16, 2017](https://reddit.com/r/femalefashionadvice/comments/65ptu4/waywt_april_16_2017/dgca3ps) | /u/saphydoodle | +118
30 | [WAYWT - February 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5rgaam/waywt_february_01_2017/dd7dgp6) | /u/shercocked | +118
31 | [WAYWT - March 12, 2017](https://reddit.com/r/femalefashionadvice/comments/5yzbvg/waywt_march_12_2017/deu7vlo) | /u/saphydoodle | +116
32 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejetqo) | /u/muzzyhoo92 | +116
33 | [WAYWT - June 04, 2017](https://reddit.com/r/femalefashionadvice/comments/6f8395/waywt_june_04_2017/dig4ttb) | /u/firstthingmonday | +115
34 | [WAYWT - April 16, 2017](https://reddit.com/r/femalefashionadvice/comments/65ptu4/waywt_april_16_2017/dgccac5) | /u/goldiehawn | +115
35 | [WAYWT - April 12, 2017](https://reddit.com/r/femalefashionadvice/comments/64z2hk/waywt_april_12_2017/dg64tqc) | /u/ayiyi | +115
36 | [WAYWT - January 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5lfogz/waywt_january_01_2017/dbva9nc) | /u/Straightgrizzly22 | +115
37 | [WAYWT - July 30, 2017](https://reddit.com/r/femalefashionadvice/comments/6qi4u7/waywt_july_30_2017/dkxkcr8) | /u/Straightgrizzly22 | +112
38 | [WAYWT - July 30, 2017](https://reddit.com/r/femalefashionadvice/comments/6qi4u7/waywt_july_30_2017/dkxu8f0) | /u/sewing_magic | +112
39 | [WAYWT - July 19, 2017](https://reddit.com/r/femalefashionadvice/comments/6o9gk9/waywt_july_19_2017/dkfyr5x) | /u/PM_ME_ITTYBITTYKITTY | +112
40 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dffkbn5) | /u/SusannaCarmichael | +112
41 | [WAYWT - March 22, 2017](https://reddit.com/r/femalefashionadvice/comments/60vi51/waywt_march_22_2017/df9kjdt) | /u/shercocked | +112
42 | [WAYWT - January 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5lfogz/waywt_january_01_2017/dbvbvwl) | /u/abexaem | +111
43 | [WAYWT - April 12, 2017](https://reddit.com/r/femalefashionadvice/comments/64z2hk/waywt_april_12_2017/dg640ls) | /u/TwoSizes | +110
44 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dffoly1) | /u/BlueSpiritBlues | +110
45 | [WAYWT - August 20, 2017](https://reddit.com/r/femalefashionadvice/comments/6uwiut/waywt_august_20_2017/dlwadct) | /u/wanderedoff | +108
46 | [WAYWT - August 09, 2017](https://reddit.com/r/femalefashionadvice/comments/6smgup/waywt_august_09_2017/dldx97t) | /u/emaleau | +108
47 | [WAYWT - May 17, 2017](https://reddit.com/r/femalefashionadvice/comments/6bpmgl/waywt_may_17_2017/dhom2ir) | /u/skyllanyx | +107
48 | [WAYWT - March 29, 2017](https://reddit.com/r/femalefashionadvice/comments/627j7s/waywt_march_29_2017/dfkebuu) | /u/DrNooooooo | +107
49 | [WAYWT - March 19, 2017](https://reddit.com/r/femalefashionadvice/comments/60aqzk/waywt_march_19_2017/df4vai9) | /u/somethings_nothings | +107
50 | [WAYWT - March 15, 2017](https://reddit.com/r/femalefashionadvice/comments/5zk88l/waywt_march_15_2017/deyqcvk) | /u/ayiyi | +107
51 | [WAYWT - July 02, 2017](https://reddit.com/r/femalefashionadvice/comments/6ktzj8/waywt_july_02_2017/djosxbk) | /u/salted_sky | +106
52 | [WAYWT - May 24, 2017](https://reddit.com/r/femalefashionadvice/comments/6d3352/waywt_may_24_2017/dhzicjz) | /u/Wasmand | +106
53 | [WAYWT - February 12, 2017](https://reddit.com/r/femalefashionadvice/comments/5tm3q4/waywt_february_12_2017/ddniypv) | /u/BlueSpiritBlues | +106
54 | [WAYWT - March 26, 2017](https://reddit.com/r/femalefashionadvice/comments/61m9z7/waywt_march_26_2017/dffmjgs) | /u/sohungy | +105
55 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejpq5h) | /u/cikayelle | +105
56 | [WAYWT - May 14, 2017](https://reddit.com/r/femalefashionadvice/comments/6b4mfm/waywt_may_14_2017/dhk4p8t) | /u/maya1987_K | +104
57 | [WAYWT - May 10, 2017](https://reddit.com/r/femalefashionadvice/comments/6adozz/waywt_may_10_2017/dhdrbqm) | /u/super_cheeky | +104
58 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8q9sw) | /u/sohungy | +104
59 | [WAYWT - February 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5rgaam/waywt_february_01_2017/dd72crh) | /u/ayiyi | +104
60 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgy7gdo) | /u/stephamelon | +103
61 | [WAYWT - August 09, 2017](https://reddit.com/r/femalefashionadvice/comments/6smgup/waywt_august_09_2017/dle4zlg) | /u/e___e | +102
62 | [WAYWT - January 22, 2017](https://reddit.com/r/femalefashionadvice/comments/5phooz/waywt_january_22_2017/dcrd132) | /u/mycatispipeyq | +102
63 | [WAYWT - July 19, 2017](https://reddit.com/r/femalefashionadvice/comments/6o9gk9/waywt_july_19_2017/dkfpjar) | /u/Lillyeth | +101
64 | [WAYWT - March 19, 2017](https://reddit.com/r/femalefashionadvice/comments/60aqzk/waywt_march_19_2017/df4y28u) | /u/BlueSpiritBlues | +101
65 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejmbo4) | /u/bean-sprout | +101
66 | [WAYWT - May 17, 2017](https://reddit.com/r/femalefashionadvice/comments/6bpmgl/waywt_may_17_2017/dhoz9xl) | /u/hiilive | +100
67 | [WAYWT - August 23, 2017](https://reddit.com/r/femalefashionadvice/comments/6vjyte/waywt_august_23_2017/dm0ztkl) | /u/evrtgurnot | +99
68 | [WAYWT - June 04, 2017](https://reddit.com/r/femalefashionadvice/comments/6f8395/waywt_june_04_2017/digcg3f) | /u/goldiehawn | +99
69 | [WAYWT - July 30, 2017](https://reddit.com/r/femalefashionadvice/comments/6qi4u7/waywt_july_30_2017/dkxpliz) | /u/e___e | +98
70 | [WAYWT - May 17, 2017](https://reddit.com/r/femalefashionadvice/comments/6bpmgl/waywt_may_17_2017/dhoi5sx) | /u/faux-kwah | +98
71 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgy9bru) | /u/babychickyellow | +97
72 | [WAYWT - February 15, 2017](https://reddit.com/r/femalefashionadvice/comments/5u89ok/waywt_february_15_2017/ddships) | /u/shercocked | +97
73 | [WAYWT - July 12, 2017](https://reddit.com/r/femalefashionadvice/comments/6mux2b/waywt_july_12_2017/dk4s5jx) | /u/shercocked | +96
74 | [WAYWT - June 21, 2017](https://reddit.com/r/femalefashionadvice/comments/6imsb7/waywt_june_21_2017/dj7jsam) | /u/shercocked | +96
75 | [WAYWT - April 26, 2017](https://reddit.com/r/femalefashionadvice/comments/67p31j/waywt_april_26_2017/dgs74zy) | /u/super_cheeky | +96
76 | [WAYWT - August 20, 2017](https://reddit.com/r/femalefashionadvice/comments/6uwiut/waywt_august_20_2017/dlw9bei) | /u/full_boyle | +95
77 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlkc2zc) | /u/Bloopity_bloop_bloop | +95
78 | [WAYWT - July 16, 2017](https://reddit.com/r/femalefashionadvice/comments/6nmxr1/waywt_july_16_2017/dkapnak) | /u/PM_ME_ITTYBITTYKITTY | +95
79 | [WAYWT - April 12, 2017](https://reddit.com/r/femalefashionadvice/comments/64z2hk/waywt_april_12_2017/dg65lnf) | /u/LentilBurguer | +95
80 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejfmyg) | /u/SusannaCarmichael | +95
81 | [WAYWT - January 29, 2017](https://reddit.com/r/femalefashionadvice/comments/5queyw/waywt_january_29_2017/dd292hu) | /u/BlueSpiritBlues | +95
82 | [WAYWT - January 25, 2017](https://reddit.com/r/femalefashionadvice/comments/5q3z3z/waywt_january_25_2017/dcw7vbg) | /u/shercocked | +95
83 | [WAYWT - April 23, 2017](https://reddit.com/r/femalefashionadvice/comments/672zjc/waywt_april_23_2017/dgnl12t) | /u/kcwashere | +94
84 | [WAYWT - April 23, 2017](https://reddit.com/r/femalefashionadvice/comments/672zjc/waywt_april_23_2017/dgn992o) | /u/niftynonsense | +94
85 | [WAYWT - August 27, 2017](https://reddit.com/r/femalefashionadvice/comments/6wcypl/waywt_august_27_2017/dm7f7us) | /u/recycledstardust | +93
86 | [WAYWT - July 02, 2017](https://reddit.com/r/femalefashionadvice/comments/6ktzj8/waywt_july_02_2017/djotysz) | /u/saphydoodle | +93
87 | [WAYWT - April 02, 2017](https://reddit.com/r/femalefashionadvice/comments/630f8l/waywt_april_02_2017/dfqgxt4) | /u/shercocked | +93
88 | [WAYWT - March 05, 2017](https://reddit.com/r/femalefashionadvice/comments/5xng3g/waywt_march_05_2017/dejnke6) | /u/ahrieru | +93
89 | [WAYWT - February 26, 2017](https://reddit.com/r/femalefashionadvice/comments/5waqux/waywt_february_26_2017/de8quxg) | /u/SusannaCarmichael | +93
90 | [WAYWT - February 12, 2017](https://reddit.com/r/femalefashionadvice/comments/5tm3q4/waywt_february_12_2017/ddnksu0) | /u/somethings_nothings | +93
91 | [WAYWT - January 08, 2017](https://reddit.com/r/femalefashionadvice/comments/5mrhi4/waywt_january_08_2017/dc5xmt3) | /u/kcwashere | +93
92 | [WAYWT - June 18, 2017](https://reddit.com/r/femalefashionadvice/comments/6i0eei/waywt_june_18_2017/dj2sd9k) | /u/bean-sprout | +92
93 | [WAYWT - February 22, 2017](https://reddit.com/r/femalefashionadvice/comments/5vjhx0/waywt_february_22_2017/de2np2z) | /u/ayiyi | +92
94 | [WAYWT - April 30, 2017](https://reddit.com/r/femalefashionadvice/comments/68g6jq/waywt_april_30_2017/dgy6owp) | /u/coffee_for_dinner | +91
95 | [WAYWT - January 18, 2017](https://reddit.com/r/femalefashionadvice/comments/5oq65a/waywt_january_18_2017/dcl6rgj) | /u/nnutella | +91
96 | [WAYWT - August 13, 2017](https://reddit.com/r/femalefashionadvice/comments/6tfwpv/waywt_august_13_2017/dlkd2kd) | /u/Lillyeth | +90
97 | [WAYWT - May 07, 2017](https://reddit.com/r/femalefashionadvice/comments/69s0je/waywt_may_07_2017/dh8yhq9) | /u/coffee_for_dinner | +90
98 | [WAYWT - April 19, 2017](https://reddit.com/r/femalefashionadvice/comments/66b4q7/waywt_april_19_2017/dgh2hkn) | /u/orlando_furiosa | +90
99 | [WAYWT - April 19, 2017](https://reddit.com/r/femalefashionadvice/comments/66b4q7/waywt_april_19_2017/dgh2u6c) | /u/tomlizzo | +90
100 | [WAYWT - March 01, 2017](https://reddit.com/r/femalefashionadvice/comments/5wwseh/waywt_march_01_2017/dedferu) | /u/ayiyi | +90

